In [7]:
# REAL TIME FACE MASK DETECTION
# AUTHOR: PRANJAL BISHT

In [8]:
import zipfile as zf
files = zf.ZipFile('Face Mask Detection.zip','r')
files.extractall()
files.close()

In [14]:
# IMPOTING THE LIBRARIES

In [15]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [16]:
# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

In [17]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [18]:
# compile and training the model

In [19]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'Face Mask Detection/Dataset/train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'Face Mask Detection/Dataset/test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

model.save('mymodel.h5',model_saved)

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
83/83 [==============================] - 37s 445ms/step - loss: 0.4825 - accuracy: 0.7702 - val_loss: 0.2076 - val_accuracy: 0.9227
Epoch 2/10
83/83 [==============================] - 31s 369ms/step - loss: 0.1594 - accuracy: 0.9452 - val_loss: 0.1004 - val_accuracy: 0.9639
Epoch 3/10
83/83 [==============================] - 28s 339ms/step - loss: 0.1178 - accuracy: 0.9574 - val_loss: 0.1011 - val_accuracy: 0.9639
Epoch 4/10
83/83 [==============================] - 27s 320ms/step - loss: 0.1073 - accuracy: 0.9635 - val_loss: 0.1032 - val_accuracy: 0.9691
Epoch 5/10
83/83 [==============================] - 27s 328ms/step - loss: 0.0945 - accuracy: 0.9635 - val_loss: 0.1018 - val_accuracy: 0.9691
Epoch 6/10
83/83 [==============================] - 28s 333ms/step - loss: 0.0848 - accuracy: 0.9688 - val_loss: 0.0721 - val_

In [23]:
mymodel=load_model('mymodel.h5')


In [26]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('Face Mask Detection/Dataset/haarcascade_frontalface_default.xml')

In [29]:
while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict_classes(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 